In [37]:
from transformers import AutoModel, BertTokenizerFast
import pandas as pd
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn.functional import cross_entropy
import numpy as np

In [38]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
train = pd.read_csv("/home/kunal/Dev/RedditBot/identifying-depression/train.csv",lineterminator='\n')
test = pd.read_csv("/home/kunal/Dev/RedditBot/identifying-depression/test.csv",lineterminator='\n')

In [40]:
train

,Unnamed: 0,text,label
0,149293,never thought id be one of those people who i...,0
1,31658,hes a racist redneck farmer judging by his fac...,0
2,119474,dmt no im not joking,0
3,223606,upliftingnews leave us to our commiserating,1
4,219628,and spend your days in bed hoping for some kin...,1
...,...,...,...
143707,11612,i havent seen my therapist in about or month...,1
143708,81389,ive been there for a couple weeks at a time bu...,1
143709,159541,the oldest is the trailblazer with restriction...,0
143710,49251,hey thanks for writing this out ive seen some...,1


In [41]:
test

,Unnamed: 0,text,label
0,233746,tommy vercetti for me,0
1,83283,youre amazing way to go i recently lost my job...,1
2,172981,happy birthday keep on living it can get better,1
3,144431,painting pft,0
4,233840,same,1
...,...,...,...
47900,179962,you said it perfectly i stay up late to be pro...,1
47901,81929,cant remember whats its called but the japanes...,0
47902,172708,lets take for instance think of it like poi...,0
47903,215792,i broke down completely twice last night and n...,1


In [42]:
df = train.append(test)
df = df.dropna()
# df
df = df[:10000]

/tmp/ipykernel_21486/3636092027.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train.append(test)


In [43]:
from sklearn.model_selection import train_test_split
  
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['label'], 
                                                                    random_state = 2021, 
                                                                    test_size = 0.3, 
                                                                    stratify = df['label'])
  
  
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state = 2021, 
                                                                test_size = 0.5, 
                                                                stratify = temp_labels)

In [44]:
pad_len = 17

In [9]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    padding=True,
    max_length = pad_len,
    truncation = True)
  
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = pad_len,
    padding= True,
    truncation = True)
  
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = pad_len,
    padding = True,
    truncation = True
)
  
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())
  
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())
  
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [10]:
#freeze the pretrained layers
for param in bert.parameters():
    param.requires_grad = False
  
#defining new layers
class BERT_architecture(nn.Module):
  
    def __init__(self, bert):
        
      super(BERT_architecture, self).__init__()
  
      self.bert = bert 
        
      # dropout layer
      self.dropout = nn.Dropout(0.2)
        
      # relu activation function
      self.relu =  nn.ReLU()
  
      # dense layer 1
      self.fc1 = nn.Linear(768,512)
        
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)
  
      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)
  
    #define the forward pass
    def forward(self, sent_id, mask):
  
      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        
      x = self.fc1(cls_hs)
  
      x = self.relu(x)
  
      x = self.dropout(x)
  
      # output layer
      x = self.fc2(x)
        
      # apply softmax activation
      x = self.softmax(x)
  
      return x

In [11]:
model = BERT_architecture(bert)

In [12]:
optimizer = AdamW(model.parameters(),lr = 0.001)


In [13]:
def train():
    
    model.train()
  
    total_loss, total_accuracy = 0, 0
    
  # empty list to save model predictions
    total_preds=[]
    
  # iterate over batches
    for step,batch in enumerate(train_dataloader):
      
    # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
              print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
  
    # push the batch to gpu
    
        batch = [r.to(device) for r in batch]
   
        sent_id, mask, labels = batch
  
    # clear previously calculated gradients 
        model.zero_grad()        
  
    # get model predictions for the current batch
        preds = model(sent_id, mask)
  
    # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)
  
    # add on to the total loss
        total_loss = total_loss + loss.item()
  
    # backward pass to calculate the gradients
        loss.backward()
  
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  
    # update parameters
        optimizer.step()
  
    # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()
  
    # append the model predictions
        total_preds.append(preds)
  
  # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
    
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
    total_preds  = np.concatenate(total_preds, axis=0)
  
  #returns the loss and predictions
    return avg_loss, total_preds

In [14]:

def evaluate():
    
  print("\nEvaluating...")
    
  # deactivate dropout layers
  model.eval()
  
  total_loss, total_accuracy = 0, 0
    
  # empty list to save the model predictions
  total_preds = []
  
  # iterate over batches
  for step,batch in enumerate(val_dataloader):
      
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
        
      # # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
              
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
  
    # push the batch to gpu
    batch = [t.to(device) for t in batch]
  
    sent_id, mask, labels = batch
  
    # deactivate autograd
    with torch.no_grad():
        
      # model predictions
      preds = model(sent_id, mask)
  
      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)
  
      total_loss = total_loss + loss.item()
  
      preds = preds.detach().cpu().numpy()
  
      total_preds.append(preds)
  
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 
  
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  
  return avg_loss, total_preds

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

device

device(type='cpu')

In [16]:
model=model.to(device)

In [17]:
from torch.utils.data import DataLoader 
train_dataloader = DataLoader(list(zip(train_seq,train_mask, train_y)), shuffle=True, batch_size=16)
val_dataloader = DataLoader(list(zip(val_seq,val_mask, val_y)), shuffle=True, batch_size=16)

In [18]:
train()

  Batch    50  of    438.
  Batch   100  of    438.
  Batch   150  of    438.
  Batch   200  of    438.
  Batch   250  of    438.
  Batch   300  of    438.
  Batch   350  of    438.
  Batch   400  of    438.


(0.6402020277497975,
 array([[-0.71609676, -0.6707124 ],
        [-0.63132036, -0.75905   ],
        [-0.63079804, -0.7596438 ],
        ...,
        [-0.11821792, -2.193752  ],
        [-0.21066636, -1.6609645 ],
        [-0.28877592, -1.3830199 ]], dtype=float32))

In [19]:
evaluate()


Evaluating...
  Batch    50  of     94.


(0.564265583106812,
 array([[-0.38390705, -1.1431748 ],
        [-0.41954044, -1.0710424 ],
        [-0.55595064, -0.8522057 ],
        ...,
        [-0.12021544, -2.1779752 ],
        [-1.3220327 , -0.3100542 ],
        [-1.4714636 , -0.2608314 ]], dtype=float32))

In [20]:
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    # print(preds.__dict__)
    preds = preds.detach().cpu().numpy()

from sklearn.metrics import classification_report
pred = np.argmax(preds, axis = 1)
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75       806
           1       0.72      0.69      0.70       694

    accuracy                           0.73      1500
   macro avg       0.73      0.73      0.73      1500
weighted avg       0.73      0.73      0.73      1500



In [23]:
torch.save(model.state_dict(), "model.pt")


In [24]:
pred

array([1, 0, 1, ..., 0, 0, 1])

In [31]:
text = "I wanna kill myself"

In [32]:
tokens = tokenizer.encode_plus(
    text,
    padding=True,
    max_length = pad_len,
    truncation = True)
tokens

{'input_ids': [101, 1045, 10587, 3102, 2870, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [33]:
tokens_seq = torch.tensor(tokens['input_ids']).unsqueeze(0)
tokens_mask = torch.tensor(tokens['attention_mask']).unsqueeze(0)

In [34]:
preds = model.forward(tokens_seq.to(device), tokens_mask.to(device))
preds = preds.detach().cpu().numpy()

pred = np.argmax(preds, axis = 1)


In [35]:
pred

array([0])

In [36]:
preds[0]

array([-0.43546766, -1.0411797 ], dtype=float32)

In [112]:
torch.save(model.state_dict(), "model.pt")
# torch.save(model.state_dict(), PATH)


In [111]:
model=model.to(device)